# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [240]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [241]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [242]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39']
['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8']
["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8']
['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8']
['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8']
['Tamba Trio', 'Logged In', 'Kaylee', 'F', '4', 'Summers', '177.18812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54

In [243]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    for line in f:
        print(line)

"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"

"Des'ree","Kaylee","F","1","Summers","246.30812","free","Phoenix-Mesa-Scottsdale, AZ","139","You Gotta Be","8"

"Mr Oizo","Kaylee","F","3","Summers","144.03873","free","Phoenix-Mesa-Scottsdale, AZ","139","Flat 55","8"

"Tamba Trio","Kaylee","F","4","Summers","177.18812","free","Phoenix-Mesa-Scottsdale, AZ","139","Quem Quiser Encontrar O Amor","8"

"The Mars Volta","Kaylee","F","5","Summers","380.42077","free","Phoenix-Mesa-Scottsdale, AZ","139","Eriatarka","8"

"Infected Mushroom","Kaylee","F","6","Summers","440.2673","free","Phoenix-Mesa-Scottsdale, AZ","139","Becoming Insane","8"

"Blue October / Imogen Heap","Kaylee","F","7","Summers","241.3971","free","Phoenix-Mesa-Scottsdale, AZ","139","Congratulations","8"

"Girl Talk","Kaylee","F","8","Summers","160.15628","free","Phoenix-Mesa-Scottsdale, AZ","139","Once again","8"

"Black Eyed Peas","Sylvie","F","0","Cruz","214.9350

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [244]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [245]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [246]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [247]:
## Query 1 - create table
query = "CREATE TABLE IF NOT EXISTS by_sessionId_and_itemInSession "
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY (sessionId, itemInSession))"
try: 
    session.execute(query)
except Exception as e:
    print(e)                    

In [248]:
## Query 1 - insert appropriate event data fields into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO by_sessionId_and_itemInSession (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"   
        
        # 0           1         2            3              4        5         6       7           8        9    10
        #"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [249]:
## Query 1
## Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = "select artist, song, length from by_sessionId_and_itemInSession where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

In [250]:
## Query 2 - create table
query = "CREATE TABLE IF NOT EXISTS by_userId_and_sessionId "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                             

In [251]:
## Query 2 - insert appropriate event data fields into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO by_userId_and_sessionId (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # 0           1         2            3              4        5         6       7           8        9    10
        #"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [252]:
## Query 2
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
query = "select artist, song, firstName, lastName from by_userId_and_sessionId where userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstName, row.lastName)

In [253]:
## Query 3 - create table
query = "CREATE TABLE IF NOT EXISTS by_song_and_user "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [254]:
# Query 3 - insert appropriate event data fields into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        print(line)
        query = "INSERT INTO by_song_and_user (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"

        # 0           1         2            3              4        5         6       7           8        9      10
        #"artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

["Des'ree", 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'You Gotta Be', '8']
['Mr Oizo', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'Flat 55', '8']
['Tamba Trio', 'Kaylee', 'F', '4', 'Summers', '177.18812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'Quem Quiser Encontrar O Amor', '8']
['The Mars Volta', 'Kaylee', 'F', '5', 'Summers', '380.42077', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'Eriatarka', '8']
['Infected Mushroom', 'Kaylee', 'F', '6', 'Summers', '440.2673', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'Becoming Insane', '8']
['Blue October / Imogen Heap', 'Kaylee', 'F', '7', 'Summers', '241.3971', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'Congratulations', '8']
['Girl Talk', 'Kaylee', 'F', '8', 'Summers', '160.15628', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'Once again', '8']
['Black Eyed Peas', 'Sylvie', 'F', '0', 'Cruz', '214.93506', 'free', 'Washington-Arlingt

In [255]:
## Query 3
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select firstName, lastName from by_song_and_user where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstName, row.lastName)

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query1 = "drop table if exists by_song_and_user"
query2 = "drop table if exists by_userId_and_sessionId"
query3 = "drop table if exists by_sessionId_and_itemInSession"
try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as e:
    print(e)     

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()